In [ ]:
!pip install portalocker>=2.0.0

In [ ]:
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2024-01-22 20:13:57.086322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 20:13:57.086442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 20:13:57.088957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 20:13:57.106825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 20:13:59.288193: W tensorflow/compiler/tf2

In [ ]:
!pip install --upgrade torchtext

In [ ]:
!pip install --upgrade pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

In [ ]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

def yield_tokens(data_iterator: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iterator:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
# UNK = unknown key
# PAD = index used for padding in order to have flexible words length
# SOS = start of sequence
# EOS = end of sequence
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    vocab_transform[language] = build_vocab_from_iterator(yield_tokens(train_iter, language),min_freq=1,specials=special_symbols, special_first=True)
    vocab_transform[language].set_default_index(UNK_IDX)

In [ ]:
def create_masks(src, tgt):
    tgt_seq_len = tgt.shape[0]
    src_seq_len = src.shape[0]

    tgt_mask = torch.triu(torch.ones((tgt_seq_len,tgt_seq_len),device = DEVICE)*float('-inf'),diagonal = 1)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).bool()

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Seq2SeqTransformer(nn.Module):
    def __init__(self,num_encoder_layers,num_decoder_layers,emb_size,nhead,src_vocab_size, tgt_vocab_size,dim_feedforward = 512,dropout = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.emb_size = emb_size
        self.transformer = Transformer(d_model=emb_size,nhead=nhead,num_encoder_layers=num_encoder_layers,num_decoder_layers=num_decoder_layers,dim_feedforward=dim_feedforward,dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = nn.Embedding(src_vocab_size, self.emb_size)
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, self.emb_size)
        self.dropout = nn.Dropout(dropout);

    def positional_embedding(self,emb_size,maxlen = 5000):
          den = torch.exp(- torch.arange(0, self.emb_size, 2)* math.log(10000) / self.emb_size)
          pos = torch.arange(0, maxlen).reshape(maxlen, 1)
          pos_embedding = torch.zeros((maxlen, self.emb_size))
          pos_embedding[:, 0::2] = torch.sin(pos * den)
          pos_embedding[:, 1::2] = torch.cos(pos * den)
          pos_embedding = pos_embedding.unsqueeze(-2)
          return pos_embedding;

    def forward(self,src,trg,src_mask,tgt_mask, src_padding_mask,tgt_padding_mask,memory_key_padding_mask):

        src_token_emb = self.src_tok_emb(src)*math.sqrt(self.emb_size)
        tgt_token_emb = self.tgt_tok_emb(trg)*math.sqrt(self.emb_size)
        pos_emb = self.positional_embedding(self.emb_size);
        src_emb = self.dropout(src_token_emb + pos_emb[:src_token_emb.size(0), :])
        tgt_emb = self.dropout(tgt_token_emb + pos_emb[:tgt_token_emb.size(0), :])
        out = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(out)

    def encode(self, src, src_mask):
        src_token_emb = self.src_tok_emb(src)*math.sqrt(self.emb_size)
        pos_emb = self.positional_embedding(self.emb_size);
        out = self.transformer.encoder(src_token_emb + pos_emb[:src_token_emb.size(0), :], src_mask)
        return out

    def decode(self, tgt, memory, tgt_mask):
        tgt_token_emb = self.tgt_tok_emb(tgt)*math.sqrt(self.emb_size)
        pos_emb= self.positional_embedding(self.emb_size);
        out = self.transformer.decoder(tgt_token_emb + pos_emb[:tgt_token_emb.size(0), :], memory,tgt_mask)
        return self.generator(out)

In [ ]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iterator, batch_size=BATCH_SIZE, collate_fn=batch_transform)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_masks(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))

In [ ]:

def evaluate(model):
    model.eval()
    losses = 0

    val_iterator = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iterator, batch_size=BATCH_SIZE, collate_fn=batch_transform)
    i = 0
    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_masks(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def add_end_and_start_tokens(token_ids):
    return torch.cat((torch.tensor([SOS_IDX]),torch.tensor(token_ids),torch.tensor([EOS_IDX])))

def transform_text_to_numbers(language, text):

    #for token in  token_transform[language](text.rstrip("\n")):
    tokens = token_transform[language](text.rstrip("\n"))
    vocab_ids = vocab_transform[language](tokens);
    text_transformed= add_end_and_start_tokens(vocab_ids)
    return text_transformed

# function to collate data samples into batch tensors
def batch_transform(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(transform_text_to_numbers(SRC_LANGUAGE, src_sample))
        tgt_batch.append(transform_text_to_numbers(TGT_LANGUAGE, tgt_sample))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
import numpy as np
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
EMB_SIZE = 512
NHEAD = 4
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
DIM_FEEDFORWARD = 1024

BATCH_SIZE = 128


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, DIM_FEEDFORWARD)

for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/MyModels/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 10#18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    torch.save(transformer, drive_path + 'transformer_dim_1024_512_model_0_'+str(epoch-1)+'.pth')

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.305, Val loss: 4.083, Epoch time = 2723.411s


KeyboardInterrupt: 

In [ ]:
torch.save(transformer, drive_path + 'transformer_dim256_model_0_final.pth')

In [ ]:
import torch
model = torch.load(drive_path+'transformer_dim256_model_0_22.pth')